# Intro to profiling and benchmarking 

In other words, 

- how long does it take my code to run?
- what sections of code are taking the longest?
- how much time are the CPUs working as opposed to waiting?
- how much memory does my program use?

Much of this Jupyter Notebook is copied from [Real Python](https://realpython.com/python-profiling/) which is a great resource for in-depth explanations of topics relating to Python. 

## Profiling - how much time?

### `time` and `timeit`

In [1]:
# time module
import time

def sleeper():
    """
    This function remains dormant without occupying the computer's CPU and allows other threads
    and programs to run
    """
    time.sleep(1.75)


def spinlock():
    """
    This function doesn't do anything but it does occupy your computer's CPU--it's 'busy waiting'
    """
    for _ in range(100_000_000):
        pass

for function in sleeper, spinlock:
    # time.perf_counter() gets the elapsed real time
    # time.process_time() gets the CPU time
    t1 = time.perf_counter(), time.process_time()
    function()
    t2 = time.perf_counter(), time.process_time()
    print(f"{function.__name__}()")
    print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
    print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
    print()

sleeper()
 Real time: 1.76 seconds
 CPU time: 0.00 seconds

spinlock()
 Real time: 0.91 seconds
 CPU time: 0.91 seconds



`time.perf_counter()` gets the elapsed real time, or wall-clock time, and `time.process_time()` get the "CPU time" (which means amount of time that a central processing unit (CPU) was used for processing instructions). These will tell you how long your functions took to execute and how much of that time they spent on the processor. If a function waits for another thread or an I/O operation to finish, then it won’t use any CPU time.



In [3]:
# timeit module
from timeit import timeit

def fib(n):
     return n if n < 2 else fib(n - 2) + fib(n - 1)



1.2 μs ± 8.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


It can be nice to use the `timeit` module in Jupyter Notebooks. The IPython kernel (which Jupyter Notebooks are built on top of) comes with some [built-in commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html#built-in-magic-commands) called "magics". A few of the [`timeit` functionalities](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) have magics. You can specify the number of times to execute the given Python code, the number of repeats, the precision in the output, etc.

In [8]:
# '%timeit' times a Python statement 
%timeit -n1 -r1 time.sleep(2)
%timeit -n2 -r1 time.sleep(1)

2.01 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


You can also use `timeit` in a Python module and via the command line (see [here](https://realpython.com/python-profiling/#timeit-benchmark-short-code-snippets))

### cProfile

In [11]:
!python -m cProfile -s tottime fibonacci.py # -s tottime sorts by the "tottime" column

         29860706 function calls (4 primitive calls) in 2.675 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
29860703/1    2.675    0.000    2.675    2.675 fibonacci.py:1(fib)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    2.675    2.675 {built-in method builtins.exec}
        1    0.000    0.000    2.675    2.675 fibonacci.py:1(<module>)




The output tell us that the program took over 2.5 seconds to finish while making exactly 29,860,706 function calls, only 4 of which were primitive or non-recursive calls, including just one non-recursive call to `fib()`. 

- ncalls: the number of calls,
- tottime: the total time spent in the given function (and excluding time made in calls to sub-functions)
- percall: the quotient of tottime divided by ncalls
- cumtime: the cumulative time spent in this and all subfunctions (from invocation until exit).
- percall: is the quotient of cumtime divided by primitive calls
- filename:lineno(function): provides the respective data of each function

## Benchmarking - how much memory?

Using [GNU Time](https://www.gnu.org/software/time/) which I installed on my Mac using Homebrew/`brew` via `brew install gnu-time` and the way to use the `brew` GNU Time is via the command `gtime`.

In [19]:
!gtime -f 'time (sec): %e\nmemory (kb): %M' python grow.py 100000

time (sec): 0.01
memory (kb): 11888


And to get a feel for how your program scales with the input "size", you can do things like

In [29]:
!for i in $(seq -f "%.0f" 1 200000 1000001); do gtime -f 'time (sec): %e\nmemory (kb): %M\n\n' python grow.py $i; done

Size = 1
time (sec): 0.00
memory (kb): 8016


Size = 200001
time (sec): 0.01
memory (kb): 15872


Size = 400001
time (sec): 0.02
memory (kb): 23712


Size = 600001
time (sec): 0.02
memory (kb): 31536


Size = 800001
time (sec): 0.03
memory (kb): 39456


Size = 1000001
time (sec): 0.04
memory (kb): 54592


